In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
dircs=os.listdir('/kaggle/input/agricultural-crops-image-classification/Agricultural-crops/')

In [33]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model

In [34]:
num_classes=len(dircs)

In [79]:
images=[]
labels=[]
for directory in dircs:
    image_paths=os.listdir(f'/kaggle/input/agricultural-crops-image-classification/Agricultural-crops/{directory}')
    for img_name in image_paths:
        images.append(f'/kaggle/input/agricultural-crops-image-classification/Agricultural-crops/{directory}/{img_name}')
        labels.append(directory)

In [80]:
df={"images":images,"labels":labels}

In [81]:
df=pd.DataFrame(df)

**Shuffling Data**

In [82]:
df = df.sample(frac=1).reset_index(drop=True)

**Encoding Labels**

In [83]:
onehot=OneHotEncoder()
lab=onehot.fit_transform(df['labels'].values.reshape(-1,1)).toarray()

In [84]:
images=df['images']

**Resizing Images**

In [85]:
imgs=[]
for img in images:
    image=cv2.imread(img)
    image=cv2.resize(image,(128,128))
    imgs.append(image)

In [86]:
images=np.array(imgs)
labels=np.array(lab)

In [87]:
x_train,x_test,y_train,y_test=train_test_split(images,labels,random_state=42,test_size=0.1)

**Normalizing Data**

In [88]:
x_train=x_train/255.0
x_test=x_test/255.0

**Using ImageDataGenerator to generate more data**

In [89]:
aug = ImageDataGenerator(rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")

**Using Pretrained VGG16**

In [119]:
model = VGG16(include_top=False,weights='imagenet', input_shape=(128, 128, 3))
for layer in model.layers:
    layer.trainable = False

flat1 = tf.keras.layers.Flatten()(model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(flat1)
model = Model(inputs=model.inputs, outputs=output)

In [120]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
metrics=["accuracy"])
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=5,
    restore_best_weights=True
)

In [121]:
H=model.fit(
x=aug.flow(x_train, y_train, batch_size=32),
validation_data=(x_test, y_test),
    batch_size=16,
epochs=50,callbacks=[callback])

Epoch 1/50
24/24 [==============================] - 4s 152ms/step - loss: 12.0503 - accuracy: 0.1180 - val_loss: 5.7116 - val_accuracy: 0.3253
Epoch 2/50
24/24 [==============================] - 3s 116ms/step - loss: 5.4615 - accuracy: 0.3525 - val_loss: 4.3935 - val_accuracy: 0.3976
Epoch 3/50
24/24 [==============================] - 3s 114ms/step - loss: 3.5881 - accuracy: 0.4544 - val_loss: 4.2117 - val_accuracy: 0.3855
Epoch 4/50
24/24 [==============================] - 3s 123ms/step - loss: 2.8519 - accuracy: 0.5375 - val_loss: 3.5687 - val_accuracy: 0.4940
Epoch 5/50
24/24 [==============================] - 3s 118ms/step - loss: 2.1990 - accuracy: 0.5858 - val_loss: 2.4061 - val_accuracy: 0.5663
Epoch 6/50
24/24 [==============================] - 3s 115ms/step - loss: 1.9302 - accuracy: 0.6327 - val_loss: 3.2762 - val_accuracy: 0.4940
Epoch 7/50
24/24 [==============================] - 3s 115ms/step - loss: 1.9841 - accuracy: 0.6434 - val_loss: 2.7343 - val_accuracy: 0.6024
Epoch

In [122]:
model.evaluate(x_test,y_test)

3/3 [==============================] - 0s 20ms/step - loss: 2.4061 - accuracy: 0.5663


[2.4061455726623535, 0.5662650465965271]

# **Validation Accuracy was not too good and if you can guide me that how can i improve it, I will be very thankful.I tried changing different parameteres but it didn't help me soo much**